In [7]:
from numba import jit, cuda
import pandas as pd
import urllib.request
import os
from PIL import Image
import numpy
import tensorflow as tf
import concurrent.futures

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
qtd_train = 4100
qtd_total = 5000
path_data = "Planilhas/NaoCaranguejo.csv"
especie = "NaoCaranguejo"

In [4]:
def download_and_process_image(url, path, size=(224, 224)):
    try:
        # Download da imagem
        urllib.request.urlretrieve(url, path)
        # Processamento da imagem
        image = Image.open(path).convert('RGB')
        image = image.resize(size)
        image.save(path, 'JPEG')
        return True
    except Exception as e:
        print(f"Erro ao processar {url}: {e}")
        return False

def salvaImagens(path_data, especie):
    data = pd.read_csv(path_data)
    data.dropna(inplace=True)
    train = data[:qtd_train]
    test = data[qtd_train:qtd_total]

    os.makedirs(f'./train2/{especie}', exist_ok=True)
    os.makedirs(f'./test2/{especie}', exist_ok=True)
    
    def process_images(dataset, folder):
        futures = []
        with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
            for i, url in enumerate(dataset['image_url']):
                path = f'{folder}/{especie}/imagem_{i}.jpg'
                futures.append(executor.submit(download_and_process_image, url, path))
            
            for future in concurrent.futures.as_completed(futures):
                if not future.result():
                    print("Um download ou processamento falhou.")
    
    process_images(train, './train2')
    process_images(test, './test2')

In [9]:
salvaImagens(path_data, especie)

Erro ao processar https://inaturalist-open-data.s3.amazonaws.com/photos/462707062/medium.jpg: <urlopen error [WinError 10054] Foi forçado o cancelamento de uma conexão existente pelo host remoto>
Um download ou processamento falhou.


In [8]:
caranguejos = ['Aratus', 
               'Callinectes', 
               'Cardisoma', 
               'Goniopsis', 
               'Leptuca', 
               'Minuca', 
               'Neohelice', 
               'Neosarmatium', 
               'Ocypode', 
               'Pachygrapsus', 
               'Scylla', 
               'Ucides']

In [9]:
caminho_principal = "./test"

for nome_caranguejo in caranguejos:
    caminho_pasta = caminho_principal + f'/{nome_caranguejo}'
    
    # Verifica se a pasta existe
    if os.path.isdir(caminho_pasta):
        # Percorre todos os arquivos da pasta
        for arquivo in os.listdir(caminho_pasta):
            # Verifica se o arquivo corresponde ao padrão "imagem_x"
            if arquivo.startswith("imagem_") and arquivo.endswith(".jpg"):  # ajuste a extensão se necessário
                # Extrai o número da imagem
                numero = arquivo.split("_")[-1]
                
                # Define o novo nome do arquivo
                novo_nome = f"{nome_caranguejo}_{numero}"
                
                # Caminhos completos para renomear
                caminho_antigo = os.path.join(caminho_pasta, arquivo)
                caminho_novo = os.path.join(caminho_pasta, novo_nome)
                
                # Renomeia o arquivo
                os.rename(caminho_antigo, caminho_novo)
    else:
        print(f"Pasta não encontrada: {caminho_pasta}")